## 予測問題の例とその役割
予測問題 (Prediction Problem) は、「固定された方策 (𝜋) に基づいて、価値関数を正確に推定する」ことが目的です。  
以下に具体的な例と、それがどのように役立つかを説明します。

### 1. 予測問題の例  
#### (1) ロボットの動作予測  
状況:  

ロボットが特定の方策 (π) に基づいて動いているとします。
方策は「障害物を避けながらランダムに動き回る」もので、変更はしない。
予測問題の目的:  

各状態でロボットが「将来的にどれくらい報酬を得られるか」を計算。  
役立つ理由:  
得られた価値関数を分析することで、方策の性能や効率を評価。
例えば、「エネルギー消費が少ない状態はどこか」を特定可能。  

### (2) ゲームのプレイデータ分析
状況:  
チェスのようなゲームで、あるプレイヤーの過去の方策 (
𝜋
π) を再現。
方策を変えずに、「その方策が特定の状態でどれだけ効果的だったか」を評価。  
予測問題の目的:  

方策に基づいて各状態 
𝑠
s の価値 
𝑉
𝜋
(
𝑠
)
V 
π
 (s) を計算。  

役立つ理由:

方策の改善をする前に、その方策の強みや弱点を定量的に評価可能。  

### (3) 気象予測システム  
状況:

システムが固定された予測モデル (
𝜋
π) に基づいて「雨が降る確率」を計算する。  
予測問題の目的:

各状態（例えば、現在の気温や湿度）において、「雨が降る可能性（価値）」を予測。  
役立つ理由:

モデルの性能を評価することで、次の予測モデルの改善につなげる。  
現行モデルの信頼性を分析。

In [1]:
import numpy as np

# 簡単な環境（5状態）
class SimpleEnv:
    def __init__(self):
        self.states = [0, 1, 2, 3, 4]
        self.terminal_states = [0, 4]
    
    def step(self, state, action):
        if state in self.terminal_states:
            return state, 0, True
        next_state = state + action
        reward = 1 if next_state == 4 else 0
        done = next_state in self.terminal_states
        return next_state, reward, done

# Semi-Gradient TDで状態価値関数を予測
def semi_gradient_td(env, policy, gamma, alpha, episodes):
    w = np.zeros(len(env.states))  # 重みベクトル
    for episode in range(episodes):
        state = 2  # 初期状態
        while True:
            action = policy[state]
            next_state, reward, done = env.step(state, action)
            # TD更新
            td_target = reward + gamma * w[next_state] if not done else reward
            td_error = td_target - w[state]
            w[state] += alpha * td_error
            if done:
                break
            state = next_state
    return w

# 方策（右に進む）
policy = {0: 0, 1: 1, 2: 1, 3: 1, 4: 0}  # 状態に応じた行動
env = SimpleEnv()

# 予測問題を解く
value_function = semi_gradient_td(env, policy, gamma=0.9, alpha=0.1, episodes=1000)
print("Learned Value Function:", value_function)


Learned Value Function: [0.  0.  0.9 1.  0. ]


### 状態 3 にいるとき、状態 2 を選ばない理由
状態 3 にいる場合、「状態 2 を選ぶ」ためには、現在の固定方策を変更する必要があります。

- 予測問題の制約
予測問題では「方策を固定」するため、状態 3 で状態 2 に戻る選択肢は考慮されません。
この学習結果は、現在の固定方策に基づいて価値関数を正確に推定することを目的としています。
- 制御問題との違い
制御問題では、状態 3 の行動を最適化し、「状態 2 に戻る方が価値が高い」と判断されれば、方策を改善する可能性があります
